# Querying FHIR Data With PartiQL Redshift

This notebook demonstrates how to use PartiQL in Redshift to analyze FHIR data stored on S3.  We will also demonstrate the use of the open source schema induction tool (https://github.com/awslabs/amazon-redshif-json-schema-induction) to generate Create Table DDL for Redshift over the JSON data.

## Step 0: Initialization

##### we will build and install the schema induction tool

In [1]:
!git clone https://github.com/awslabs/amazon-redshift-json-schema-induction.git


Cloning into 'amazon-redshift-json-schema-induction'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 103 (delta 29), reused 90 (delta 22), pack-reused 0
Receiving objects: 100% (103/103), 2.29 MiB | 17.73 MiB/s, done.
Resolving deltas: 100% (29/29), done.


In [9]:
!chmod a+x ./amazon-redshift-json-schema-induction/scripts/install_mvn.sh
!sudo ./amazon-redshift-json-schema-induction/scripts/install_mvn.sh > install_maven.log

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9283k  100 9283k    0     0  6402k      0  0:00:01  0:00:01 --:--:-- 6398k


In [11]:
# this command will take few min to build the schema induction tool
!cd amazon-redshift-json-schema-induction && /opt/maven/bin/mvn package > build.log

In [12]:
!cp amazon-redshift-json-schema-induction/schema-induction/target/*.jar /home/ec2-user/SageMaker

## Step 1:  Download the FHIR data and store it in S3

In [13]:
import random
import string
import json
def randomString(stringLength=10):
    """Generate a random string of fixed length """
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(stringLength))

bucket_name = "demo-partiql-" + randomString()
bucket_name

'demo-partiql-zutfcihjex'

In [14]:
!aws s3 mb s3://$bucket_name

make_bucket: demo-partiql-zutfcihjex


In [19]:
!ls -alh ./amazon-redshift-json-schema-induction/data/fhir/claims.json

-rw-rw-r-- 1 ec2-user ec2-user 16M Feb  3 19:30 ./amazon-redshift-json-schema-induction/data/fhir/claims.json


In [20]:
# The claims json has no line breaks. We will show 1000 characters to not overwhelm the browser
!head -c 1000 ./amazon-redshift-json-schema-induction/data/fhir/claims.json

[{"resourceType": "Claim", "id": "95644393-f809-4267-9cad-1189c7321e9f", "status": "active", "use": "complete", "patient": {"reference": "Patient/d3af67c9-0c02-45f2-bc91-fea45af3ee83"}, "billablePeriod": {"start": "2003-07-21T01:02:35+04:00", "end": "2003-07-21T01:32:35+04:00"}, "organization": {"reference": "Organization/ed410eaa-33e7-4b26-abb5-35416b9a6495"}, "diagnosis": [{"sequence": 1, "diagnosisReference": {"reference": "Condition/00558851-32c7-4457-833a-13fa0e484683"}}], "item": [{"sequence": 1, "encounter": [{"reference": "Encounter/a9d7304a-9c11-496c-8a92-2573aa4ce1a7"}]}, {"sequence": 2, "diagnosisLinkId": [1]}], "total": {"value": 125.0, "system": "urn:iso:std:iso:4217", "code": "USD"}}, {"resourceType": "Claim", "id": "c8f19946-23c8-4c53-8f7b-38841ee314da", "status": "active", "use": "complete", "patient": {"reference": "Patient/d3af67c9-0c02-45f2-bc91-fea45af3ee83"}, "billablePeriod": {"start": "2009-07-20T01:02:35+04:00", "end": "2009-07-20T01:32:35+04:00"}, "organization

In [21]:
!aws s3 cp ./amazon-redshift-json-schema-induction/data/fhir/claims.json s3://$bucket_name/fhir/claims/claims.json

upload: amazon-redshift-json-schema-induction/data/fhir/claims.json to s3://demo-partiql-zutfcihjex/fhir/claims/claims.json


## Step 2:  Download the Schema Induction Tool and run it for the data above

In [22]:
!ls -alh *.jar

-rw-rw-r-- 1 ec2-user ec2-user 310K Feb  3 19:33 original-schema-induction-1.0.0.jar
-rw-rw-r-- 1 ec2-user ec2-user 154M Feb  3 19:33 schema-induction-1.0.0.jar


In [23]:
!java -jar schema-induction-1.0.0.jar -h

Usage: <main class> [-ah] [-c=<s3>] [-d=<outDdlFile>] -i=<inputFile>
                    [-l=<tableLocation>] [-r=<region>] [-root=<rootDefinition>]
                    [-s=<outSchemaFile>] [--stats=<outStatsFile>]
                    [-t=<tableName>]
  -a, --array               is the document a json array
  -c, --cred=<s3>           which type of s3 credentials to use (ec2|profile)
  -d, --ddl=<outDdlFile>    An output ddl file for Redshift
  -h, --help                display a help message
  -i, --input=<inputFile>   An input json file path.
  -l, --location=<tableLocation>
                            table location to use when creating DDL
  -r, --region=<region>     s3 region to use
      -root, --root=<rootDefinition>
                            s3 region to use
  -s, --schema=<outSchemaFile>
                            An output schema file json file path
      --stats=<outStatsFile>
                            An output stats file
  -t, --table=<tableName>   table name to use w

### Now lets run the tool 

In [24]:
%%bash -s $bucket_name --out output --err error

java -jar schema-induction-1.0.0.jar \
-i s3://$1/fhir/claims/claims.json \
-d claims.ddl \
-t fhir.Claims \
-l s3://$1/fhir/claims \
-r "us-east-2"  \
-a \
-s claims.schema.json \
-root Claim

In [25]:
print("output:",output,"error:", error)

output: 0    [main] INFO  aws.json.schema.induction.Cli  – total number of unique paths: 45
89   [main] WARN  com.networknt.schema.JsonMetaSchema  – Unknown keyword discriminator - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
 error: 


In [26]:
# Lets review the generated DDL
!cat claims.ddl

create external table fhir.Claims(
	"resourceType" varchar(6),
	"id" varchar(43),
	"status" varchar(7),
	"use" varchar(9),
	"patient" struct<"reference": varchar(52)>,
	"billablePeriod" struct<"start": varchar(30),"end": varchar(30)>,
	"organization" struct<"reference": varchar(58)>,
	"diagnosis" array<struct<"sequence": double precision,"diagnosisReference": struct<"reference": varchar(55)>>>,
	"item" array<struct<"sequence": double precision,"encounter": array<varchar(55)>,"diagnosisLinkId": array<double precision>,"informationLinkId": array<double precision>,"net": struct<"value": double precision,"system": varchar(24),"code": varchar(3)>,"procedureLinkId": array<double precision>>>,
	"total" struct<"value": double precision,"system": varchar(24),"code": varchar(3)>,
	"information" array<struct<"sequence": double precision,"category": struct<"coding": array<struct<"system": varchar(52),"code": varchar(4)>>>,"valueReference": struct<"reference": varchar(58)>>>,
	"procedure" array<str

In [27]:
# Lets display the induced schema for Claims
from IPython.display import JSON
schema = json.load(open('claims.schema.json'))
JSON(schema)

<IPython.core.display.JSON object>